In [1]:
from Bio import Entrez
import urllib.request
import urllib
import pandas as pd
import numpy as np

In [2]:
# TODO: paginate to get more results (set at 1000 for testing, max is 10000)
def search(query):
    Entrez.email = 'your.email@example.com'
    handle = Entrez.esearch(db='pubmed', 
                            sort='relevance', 
                            retmax='1000',
                            retmode='xml', 
                            term=query)
    results = Entrez.read(handle)
    return results

In [3]:
def search_and_count(query):
    results = search(query)
    if results['IdList']:
        handle = Entrez.esummary(db="pubmed", id=','.join(results['IdList']), retmode="xml")
        records = Entrez.parse(handle)
        return(len(list(records)))
    else:
        return(0)

In [4]:
#Get MESH FILE USING THIS URL AND SAVE AS mesh-specialties.csv
#https://id.nlm.nih.gov/mesh/query?query=PREFIX+rdf%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22-rdf-syntax-ns%23%3E%0D%0APREFIX+rdfs%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E%0D%0APREFIX+xsd%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2001%2FXMLSchema%23%3E%0D%0APREFIX+owl%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0D%0APREFIX+meshv%3A+%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2Fvocab%23%3E%0D%0APREFIX+mesh%3A+%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2F%3E%0D%0APREFIX+mesh2015%3A+%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2F2015%2F%3E%0D%0APREFIX+mesh2016%3A+%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2F2016%2F%3E%0D%0APREFIX+mesh2017%3A+%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2F2017%2F%3E%0D%0A%0D%0A+SELECT+DISTINCT+%3Fdescriptor+%3Flabel+%3FtreeNum+%3FmedicineTreeNum%0D%0A+FROM+%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%3E%0D%0A%0D%0A+WHERE+%7B%0D%0A+++mesh%3AD008511+meshv%3AtreeNumber+%3FmedicineTreeNum+.%0D%0A+++%3FtreeNum+meshv%3AparentTreeNumber+%3FmedicineTreeNum+.%0D%0A+++%3Fdescriptor+meshv%3AtreeNumber+%3FtreeNum+.%0D%0A+++%3Fdescriptor+rdfs%3Alabel+%3Flabel+.%0D%0A+%7D%0D%0A+%0D%0AORDER+BY+%3FchildTreeNum%0D%0A&format=CSV&inference=true&year=current&limit=1000&offset=0#lodestart-sparql-results



In [5]:
MAIN_SPECIALTY_TREE_LENGTH = 11
mesh_specialties = pd.read_csv('mesh-specialties.csv')
mesh_specialties = mesh_specialties.drop(columns="medicineTreeNum")
mesh_specialties = mesh_specialties.applymap(lambda x : x.replace("http://id.nlm.nih.gov/mesh/",""))
mesh_specialties = mesh_specialties.sort_values('treeNum')
mesh_specialties['mainSpecialty'] = mesh_specialties['treeNum'].str.slice(0,MAIN_SPECIALTY_TREE_LENGTH)
mesh_specialties['level'] = (mesh_specialties['treeNum'].str.len() - MAIN_SPECIALTY_TREE_LENGTH) // 4
mesh_specialties[:15]

,descriptor,label,treeNum,mainSpecialty,level
126,D000073316,Addiction Medicine,H02.403.007,H02.403.007,0
15,D000295,Adolescent Medicine,H02.403.014,H02.403.014,0
53,D000337,Aerospace Medicine,H02.403.029,H02.403.029,0
54,D000486,Allergy and Immunology,H02.403.044,H02.403.044,0
101,D007120,Immunochemistry,H02.403.044.500,H02.403.044,1
59,D000776,Anesthesiology,H02.403.066,H02.403.066,0
66,D052938,Bariatric Medicine,H02.403.074,H02.403.074,0
60,D001524,Behavioral Medicine,H02.403.090,H02.403.090,0
79,D015510,Clinical Medicine,H02.403.200,H02.403.200,0
3,D019317,Evidence-Based Medicine,H02.403.200.400,H02.403.200,1


**Note:** Some specialties appear several times because they are multi-axial (e.g., Neurotology). https://meshb.nlm.nih.gov/record/ui?ui=D063165

In [6]:
mesh_specialties[mesh_specialties.label=="Neurotology"]

,descriptor,label,treeNum,mainSpecialty,level
67,D063165,Neurotology,H02.403.600.500,H02.403.600,1
68,D063165,Neurotology,H02.403.810.526.500,H02.403.810,2


In [7]:
specialties_set = set(mesh_specialties.label.to_list())
len(specialties_set) # Means there are 10 duplicates

119

In [8]:
main_specialties = mesh_specialties[mesh_specialties.level==0]
main_specialties = main_specialties.drop(columns=["descriptor","mainSpecialty","level"])
main_specialties['query_eng'] = main_specialties['label'] + "[MH] AND ENG[LA]"
main_specialties['query_spa'] = main_specialties['label'] + "[MH] AND SPA[LA]"
main_specialties[:15]

,label,treeNum,query_eng,query_spa
126,Addiction Medicine,H02.403.007,Addiction Medicine[MH] AND ENG[LA],Addiction Medicine[MH] AND SPA[LA]
15,Adolescent Medicine,H02.403.014,Adolescent Medicine[MH] AND ENG[LA],Adolescent Medicine[MH] AND SPA[LA]
53,Aerospace Medicine,H02.403.029,Aerospace Medicine[MH] AND ENG[LA],Aerospace Medicine[MH] AND SPA[LA]
54,Allergy and Immunology,H02.403.044,Allergy and Immunology[MH] AND ENG[LA],Allergy and Immunology[MH] AND SPA[LA]
59,Anesthesiology,H02.403.066,Anesthesiology[MH] AND ENG[LA],Anesthesiology[MH] AND SPA[LA]
66,Bariatric Medicine,H02.403.074,Bariatric Medicine[MH] AND ENG[LA],Bariatric Medicine[MH] AND SPA[LA]
60,Behavioral Medicine,H02.403.090,Behavioral Medicine[MH] AND ENG[LA],Behavioral Medicine[MH] AND SPA[LA]
79,Clinical Medicine,H02.403.200,Clinical Medicine[MH] AND ENG[LA],Clinical Medicine[MH] AND SPA[LA]
30,Community Medicine,H02.403.220,Community Medicine[MH] AND ENG[LA],Community Medicine[MH] AND SPA[LA]
0,Dermatology,H02.403.225,Dermatology[MH] AND ENG[LA],Dermatology[MH] AND SPA[LA]


In [9]:
for index, specialty in main_specialties.iterrows():
    print(specialty['label'], "-", search_and_count(specialty['query_eng']), "-", search_and_count(specialty['query_spa']))

Addiction Medicine - 43 - 0
Adolescent Medicine - 1000 - 24
Aerospace Medicine - 1000 - 50
Allergy and Immunology - 1000 - 242
Anesthesiology - 1000 - 736
Bariatric Medicine - 63 - 0
Behavioral Medicine - 1000 - 4
Clinical Medicine - 1000 - 1000
Community Medicine - 1000 - 209
Dermatology - 1000 - 583
Disaster Medicine - 643 - 5
Emergency Medicine - 1000 - 71
Forensic Medicine - 1000 - 1000
General Practice - 1000 - 1000
Genetics, Medical - 1000 - 497
Geography, Medical - 1000 - 56
Geriatrics - 1000 - 687
Global Health - 1000 - 728
Hospital Medicine - 129 - 3
Integrative Medicine - 1000 - 6
Internal Medicine - 1000 - 1000
Military Medicine - 1000 - 259
Molecular Medicine - 1000 - 40
Naval Medicine - 1000 - 100
Neurology - 1000 - 820
Osteopathic Medicine - 1000 - 10
Palliative Medicine - 227 - 1
Pathology - 1000 - 1000
Pediatrics - 1000 - 1000
Physical and Rehabilitation Medicine - 1000 - 314
Psychiatry - 1000 - 1000
Public Health - 1000 - 1000
Radiology - 1000 - 1000
Regenerative Medic